In [1]:
import pandas as pd
import numpy as np
import keras
import statistics as st
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.utils import to_categorical 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from matplotlib import pyplot
from sklearn.utils import class_weight

Using TensorFlow backend.


# Wstęp

Celem niniejszego raportu jest analiza zdolności predykcyjnych sieci neuronowych w problemie klasyfikacji, a także porównanie ich z możliwościami regresji logistycznej. Możliwości dyskryminacyjne algorytmu zostaną sprawdzone za pomocą odpowiednich wskaźników jakości. Analiza ta zostanie przeprowadzona na zbiorze danych pochodzącym z badania "Diagnoza Społeczna. Warunki i jakość życia Polaków". Poniższa analiza pozwoli odpowiedzieć na pytanie, jak dobrze model sieci neuronowych będzie przewidywać wynik zmiennej objaśnianej na podstawie ustalonych zmiennych objaśniających oraz który z wymienionych algorytmów posiada lepsze zdolności dyskryminacyjne na wybranym zbiorze danych
    

# Opis i eksploracja zbioru danych

Finalny zbiór danych złożony jest z dwóch części. Pierwsza część to zbiór który odnosi się do danych dotyczących osób, które są głowami gospodarstw domowych i brały udziały w fali z roku 2015. Druga część dotyczy gospodarstw domowych które również brały udział w fali z roku 2015. Te dwie części zostały połączone za pomocą numeru gospodarstwa domowego, który jest zmienną z unikalnymi wartościami występującą w obu zbiorach. Obserwacje z brakami danych zostały usunięte ze zbioru.

Poniższa analiza danych została wykonana wcześniej - w ramach pracy licencjackiej.

### Zmienne użyte w modelu

Zmienna objaśniana w modelu odnosi się do subiektywnej oceny sytuacji finansowej gospodarstwa domowego (zmienna fin_sit). Jest to zmienna binarna która przyjmuje dwie wartości: 
* dobra subiektywna sytuacja finansowa - wartość równa 0
* zła subiektywna sytuacja finasowa - wartość równa 1


Wynik                                         | N             | % 
----------------------------------------- | ------------- |-------
Dobra subiektywna sytuacja finansowa (0)  | 3171          | 33%
Zła subiektywna sytuacja finansowa (1)    | 6426          | 67%

Liczba obserwacji z kompletnym źródłem infromacji w zbiorze danych wynosi 9597. 


Wyróżniono kilka zmiennych które mają istotny wpływ na subiektywną ocenę sytuacji finansowej:


Wiek głowy rodziny (zmienna wiek):
1. Poniżej 34 lat
2. Pomiędzy 35 a 44 lat
3. Pomiędzy 45 a 59 lat
4. Pomiędzy 60 a 64 lata
5. Powyżej 65 lat

Biologiczny typ rodziny (zmienna biological_type):
1. Małżeństwo bez dzieci
2. Małżeństwo z 1 dzieckiem
3. Małżeństwo z 2+ dziećmi
4. Nierodzinne jednoosobowe
5. Rodziny niepełne

Poziom edukacji głowy rodziny (zmienna education):
1. Podstawowe i niższe
2. Zasadnicze zawodowe / gimnazjum
3. Średnie
4. Wyższe i poliecalne

Miejsce zamieszkania (zmienna place_residence):
1. Wieś
2. Miasto



Status socjoekonomiczny (zmienna soceco_group):
1. gospodarstwa pracowników 
2. gospodarstwa emerytów 
3. gospodarstwa pracujących na własny rachunek 
4. gospodarstwa rolników 
5. gospodarstwa rencistów

Liczba osób w gospodarstwie domowym (zmienna num_of_ind)



|Nazwa zmiennej                                     | Wynik zmiennej                                                      |       Czy dobra subiektywna ocena sytuacji finansowej| |
|-----------------------------------------|--------------------------------------|-----------------|------|
|                                        |                                     | Yes                            | No  |
|                                        |                                     |                                |     |
| Age of head of the family              | Below 34                            | 9%                             | 6%  |
|                                        | Between 35 and 44                   | 19%                            | 13% |
|                                        | Between 45 and 59                   | 30%                            | 30% |
|                                        | Between 60 and 64                   | 11%                            | 14% |
|                                        | Above 65+                           | 31%                            | 37% |
|                                        |                                     |                                |     |
| Biological type of households          | Marriage without children           | 34%                            | 23% |
|                                        | Marriage with 1 child               | 20%                            | 16% |
|                                        | Marriage with 2+ children           | 22%                            | 22% |
|                                        | Non-family one-person               | 18%                            | 25% |
|                                        | One-parent families                 | 6%                             | 14% |
|                                        |                                     |                                |     |
| Education level of head of the family  | Primary and lower education         | 8%                             | 23% |
|                                        | Vocational education                | 24%                            | 35% |
|                                        | Secondary education                 | 30%                            | 27% |
|                                        | Higher education                    | 38%                            | 15% |
|                                        |                                     |                                |     |
| Class of place of residence            | Rural area                          | 31%                            | 45% |
|                                        | Town                                | 69%                            | 55% |
|                                        |                                     |                                |     |
| Socio-economic group                   | Employed                            | 51%                            | 41% |
|                                        | Pensioners                          | 35%                            | 40% |
|                                        | Self-Employed                       | 8%                             | 3%  |
|                                        | Farmers                             | 4%                             | 7%  |
|                                        | Disability pensioners               | 2%                             | 9%  |
|                                        |                                     |                                |     |
| Number of individuals in household     | 1                                   | 18%                            | 25% |
|                                        | 2                                   | 36%                            | 30% |
|                                        | 3                                   | 21%                            | 19% |
|                                        | 4                                   | 18%                            | 16% |
|                                        | 5                                   | 5%                             | 7%  |
|                                        | 6+                                  | 2%                             | 3%  |

W każdej ze zmiennych widzimy istotne różnice pomiędzy wynikiem zmiennej objaśnianej w poszczególnych grupach. Najmniejsze różnice wystąpiły w zmiennej dotyczącej wieku w grupie Pomiędzy "45 a 59 lat", a także w zmiennej odnoszącej się do edukacji głowy rodziny w grupie "Średnie". Jednakże całościowo zmienne te mają istotny wpływ na różnicowanie zmiennej objaśnianej.  

In [2]:
# Wczytanie zbioru danych
dataframe = pd.read_csv('data2.csv')
mean = st.mean(dataframe['num_of_ind'])
sd = st.stdev(dataframe['num_of_ind'])

#normalizacja zmiennej
dataframe['num_of_ind_norm'] =  (dataframe['num_of_ind'] - mean) / sd
dataframe =dataframe[['fin_sit', 'soceco_group','biological_type', 'education', 'place_residence', 'wiek', 'num_of_ind', 'num_of_ind_norm']]

dataframe.head()

,fin_sit,soceco_group,biological_type,education,place_residence,wiek,num_of_ind,num_of_ind_norm
0,1,_farmers,one_parent,primary&lower,rural_area,Above 65+,2,-0.450468
1,1,_dis_pensioners,marr_no_child,primary&lower,rural_area,Between 60 and 64,2,-0.450468
2,1,_employed,one_person,secondary,town,Between 45 and 59,1,-1.179989
3,0,_self-employed,marr_no_child,secondary,town,Between 45 and 59,2,-0.450468
4,0,_self-employed,marr_2+_child,secondary,town,Between 35 and 44,4,1.008574


In [3]:
# Stworzenie obserwacji zero-jedynkowych w zmiennych nominalnych lub porządkowych
a= pd.get_dummies(dataframe[['soceco_group', 'biological_type',  'education', 'place_residence', 'wiek']], prefix='')

df = pd.concat([dataframe[['fin_sit', 'num_of_ind_norm']].reset_index(drop=True), a], axis=1)

#Podzial na zmienne objasniajace i zmienna objasniana
X = df.iloc[:, 1:25]
y = df.iloc[:, 0]

df.head()

,fin_sit,num_of_ind_norm,__dis_pensioners,__employed,__farmers,__pensioners,__self-employed,_marr_1_child,_marr_2+_child,_marr_no_child,...,_primary&lower,_secondary,_vocational,_rural_area,_town,_Above 65+,_Below 34,_Between 35 and 44,_Between 45 and 59,_Between 60 and 64
0,1,-0.450468,0,0,1,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,1,-0.450468,1,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,1
2,1,-1.179989,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
3,0,-0.450468,0,0,0,0,1,0,0,1,...,0,1,0,0,1,0,0,0,1,0
4,0,1.008574,0,0,0,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0


# Regresja logistyczna

Model regresji logistycznej został zbudowany na wyżej opisanych danych, z wykorzystaniem wszystkich zmiennych. Model ten był przedmiotem mojej pracy licencjackiej, do której wyników będę się odnosił.

Wszystkie zmienne w modelu były istotne, nie występowała współliniowość zmiennych objaśniających, a wyniki testów zgodności wskazywały na dobre dopasowanie modelu do danych. 

# Opis architektury sieci i procedury uczenia

### Podział zbioru danych 

Głównym celem budowy modeli predykcyjnych jest późniejsze wykorzystanie przewidywań modelu na danych niedostępnych podczas procesu uczenia. W celu uniknięcia przeszacowania, czyli nadmiernego optymizmu dotyczącego możliwości dyskryminacyjnych modelu, zbiór danych podzielono na dane treningowe i testowe. Do danych treningowych zostało losowo przypisane 80% obserwacji, natomiast do danych testowych 20%.   

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=0)

### Model sieci neuronowych

Do predykcji zmiennej objaśnianej, czyli subiektywnej oceny sytuacji finansowej gospodarstwa domowego zostanie zastosowany model dyskryminatywnych głębokich sieci neuronowych. Jest to model, który pozwala na dyskryminacje wzorców dla celów ich klasyfikacji, należy on do modeli stosowanych do nadzorowanego uczenia się. Model ten zwracał będzie prawdopodobieństwo wystąpienia złej subiektywnej oceny sytuacji finansowej. Na tej podstawie będzie dokonywana predykcja i ocena siły dyskryminacyjnej modelu sieci neuronowych.

### Funkcje aktywacji

Funkcje aktywacji są bardzo ważną częścią sieci neuronowych. Właśności sieci zmieniają się w zależności od tego jakie funkcje aktywacji wybierzemy.

#### Funkcje aktywacji w ukrytych warstwach
Funkcją aktywacji która została wybrana w każdej ukrytej warstwie była funkcja ReLU, która pozwala uniknąć problemu zanikającego gradientu. Jest ona szeroko wykorzystywana, ponieważ jest bardzo efektywna w wielu typach sieci neuronowych.
Jeśli w sieci występują małe gradienty oraz posiadamy wiele ukrytych warstw, oznacza to że gradienty te będą przez siebie mnożone wiele razy podczas propagacji wstecznej. Komputery mają pewne ograniczenia dotyczące pracy z liczbami, jeśli pomnożymy wiele bardzo małych liczb, wartość gradientu bardzo szybko spadnie i zaniknie.

#### Finalna funkcja aktywacji
W problemach binarnej klasyfikacji najczęściej wykorzystywaną funkcją finalną jest funkcja Sigmoid. Zwaraca ona prawdopodobieństwo wystąpienia klasy pozytywnej.


### Funkcja straty

Dla problemów klasyfikacji binarnej najczęściej używaną i najbardziej efektywną funkcją straty jest entropia krzyżowa. 

Entropia krzyżowa liczy wynik, który podsumowuje średnią wartość różnicy pomiędzy rzeczywistym oraz przewidywanym rozkładem prawdopodobieństwa predykcji klasy pozytywnej. Wynik ten jest minimalizowany, a idealny model ma stratę krzyżową entropii równą 0.

Sieci neuronowe stworzone do klasyfikacji, które używają funkcji sigmoidalnej w warstwie wyjściowej uczą się szybciej wykorzystując funkcję entropii krzyżowej niż błąd średniokwadratowy.

### Algorytm optymalizacyjny

Wybór algorytmu optymalizacyjnego jest jednym z najważniejszych kroków w trakcie tworzenia sieci neuronowych. Określa on w jaki sposób dobierane są parametry sieci. 

W stworzonym modelu wykorzystany został algorytm ADAM (Adaptive Moment Estimation), który jest rozszerzeniem algorytmu SGD (Stochastic Gradient Descent). 

Algorytm ADAM jest popularny w dziedzinie deep learningu, ponieważ szybko osiąga dobre rezultaty. W praktyce ADAM jest rekomendowany jako domyślny algorytm do uczenia sieci, ponieważ wypada zwykle lepiej w porównaniu do innych algorytmów optymalizacyjnych. Wnioski te zostały sprawdzone w modelu. Okazało się, że algorytm ADAM daje bardzo zbliżone do innych algorytmów rezultaty. Finalnie właśnie ten algorytm został zaimplementowany w sieci.

### Strojenie hiperparametrów

Problem dopasowania głębokich sieci neuronowych (regularization of deep
learning) jest bardzo istotny, gdyż sieci wielowarstwowe potrafią bardzo
dobrze dopasować się do danych uczących (tzw. problem zbytniego dopasowania
się – overfitting problem), lecz na danych testowych mogą zachowywać się
niepoprawnie. Im więcej warstw tym potencjalnie tworzone są bardziej
skomplikowane funkcje nieliniowe, które pogłębiają ten problem.


### Architerktura sieci neuronowych

Wybór liczby ukrytych wartsw i liczby neuronów w każdej warstwie nie jest łatwym zadaniem. Chcemy wybrać na tyle dużą arkitekturę sieci, aby poprawnie aproksymować zadaną funkcję, ale nie na tyle dużą aby zajmowało to zbyt dużo czasu. Ponadto głębokie sieci neuronowe wymagają dużej ilości danych do treningu. Oznacza to, że nie osiągniemy dużej korzyści na dodawaniu kolejnych wartsw i neuronów, gdy liczba obserwacji w zbiorze treningowym nie jest duża. Okazuje się również, że zbyt głębokie i rozbudowane sieci neuronowe mają tendencję do przeuczenia. 

W związku tym liczba ukrytych wartsw oraz liczba neuronów została wybrana przy uwzględnieniu wyżej wspomnianych własności. Liczba ukrytych warstw w modelu wynosi 2, liczba neuronów w pierwszej warstwie wynosi 15, a w drugiej 8.

### Batch size i epochs

Liczba epochs określa ile razy algorytm użyje wszystkich obserwacji zawartych w zbiorze uczącym, natomiast  rozmiar paczki danych (batch size) definiuje liczbę próbek, którą musi przejść sieć przed aktualizacją wewnętrznych parametrów. W ogólnym rozumieniu jest to pętla, w której dokonywane są prognozy porównywane z oczekiwanymi zmiennymi wyjściowymi. Na końcu pętli obliczany jest błąd, który służy sieci do ulepszania modelu.

Nie ma ściśle określonych reguł wyboru liczby epok i rozmiaru paczki danych. Nie ma gwarancji, że użycie większej liczby epok przyniesie lepsze rezultaty. Dlatego oba parametry zostaną wybrane metodą prób i błędów. 

W zaproponowanym modelu liczba epok wynosi 40, a batch size wynosi 32.

In [5]:
model = Sequential()
model.add(Dense(15, activation='relu', input_dim=22))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='ADAM', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [6]:
history = model.fit(X_train, y_train, epochs=40, batch_size = 32)

Epoch 1/40
7677/7677 [==============================] - 1s 149us/step - loss: 0.5995 - accuracy: 0.6708
Epoch 2/40
7677/7677 [==============================] - 1s 65us/step - loss: 0.5799 - accuracy: 0.6908
Epoch 3/40
7677/7677 [==============================] - 1s 68us/step - loss: 0.5732 - accuracy: 0.7054
Epoch 4/40
7677/7677 [==============================] - 0s 63us/step - loss: 0.5693 - accuracy: 0.7029
Epoch 5/40
7677/7677 [==============================] - 0s 64us/step - loss: 0.5672 - accuracy: 0.7086
Epoch 6/40
7677/7677 [==============================] - 0s 64us/step - loss: 0.5652 - accuracy: 0.7106
Epoch 7/40
7677/7677 [==============================] - 0s 63us/step - loss: 0.5649 - accuracy: 0.7100
Epoch 8/40
7677/7677 [==============================] - 0s 64us/step - loss: 0.5637 - accuracy: 0.7106
Epoch 9/40
7677/7677 [==============================] - 0s 64us/step - loss: 0.5633 - accuracy: 0.7106
Epoch 10/40
7677/7677 [==============================] - 0s 64us/step - 

### Younden index - wybór cut-off point 

Zbudowany model sieci neuronowych zwraca prawdopodobieństwo wystąpienia klasy pozytywnej, dlatego ważne jest wybrać odpowiedni cut-off, który określi dla jakiego prawdopodobieństwa dana obserwacja zostanie przypisana do klasy pozytywnej, a dla jakiego do klasy negatywnej.

Często wykorzystywaną metodą wyboru punktu odcięcia dla predykcji zmiennej objasnianej jest Yonden index, który określa optymalny cut-off, który maksymalizuje Younden function na przestrzeni możliwych punktów odcięcia (c). Younden jest funkcją sensitivity (Se) i specificity (Sp). 

 <center>J(c) = Se(c) + Sp(c) - 1</center>

In [7]:
pred_test = model.predict(X_test)
a  = np.linspace(start=0, stop=1, num=101)

#Predykcja na podstawie wybranego cut-off point
y_pred =  []
for x in a:
    abc = np.where(pred_test[:,] > x, 1, 0)
    y_pred.append(abc)


    
#obliczenie sensitivity i specificity dla danego cut-off point    
accuracy = []
sensitiviy = []
specificity = []


d = np.linspace(start = 0, stop = 100, num = 101).astype(int)
for x in d:
    tn, fp, fn, tp = confusion_matrix(np.array(y_test), y_pred[x]).ravel()
    accuracy1 = (tp + tn) / (tp + tn +fn+fp)
    sensitiviy1 = tp / (tp +fn) 
    specificity1 = tn / (tn+fp)
    
    accuracy.append(accuracy1)
    sensitiviy.append(sensitiviy1)
    specificity.append(specificity1)   
    
#Younden index jako pomoc przy wyborze najoptymalniejszego cut-off point    
Younden_index = []

for x in d:
    Younden_index1 = sensitiviy[x] + specificity[x] - 1
    
    Younden_index.append(Younden_index1)

In [8]:
index_max = Younden_index.index(np.max(Younden_index))
dp_auc = roc_auc_score(np.array(y_test), y_pred[index_max])
print('Wybrane statystyki - model sieci neuronowych')
print('Cut-off:', "0.", index_max)
print('Accuracy:', round(accuracy[index_max], 2))
print('Sensitiviy:', round(sensitiviy[index_max],2))
print('Specificity:', round(specificity[index_max],2))
print('AUC', round(dp_auc,2))

print('Wybrane statystyki - model regresji liniowej')
print('Cut-off:', 0.63)
print('Accuracy:', 0.68)
print('Sensitiviy:', 0.75)
print('Specificity:', 0.56)
print('AUC', 0.69)

Wybrane statystyki - model sieci neuronowych
Cut-off: 0. 67
Accuracy: 0.66
Sensitiviy: 0.66
Specificity: 0.65
AUC 0.66
Wybrane statystyki - model regresji liniowej
Cut-off: 0.63
Accuracy: 0.68
Sensitiviy: 0.75
Specificity: 0.56
AUC 0.69


### Podsumowanie

Zbudowany model sieci neuronowych posiada zbliżone możliwości dyskryminacyjne do modelu regresji logisycznej w przypadku predykcji zmiennej objaśnianej dotyczącej subiektywnej sytuacji finansowej gospodarstw domowych na podanym zbiorze danych. Wystąpiły minimalne różnice w wartościach statystyk oceniających trafność predykcji obu modeli klasyfikacyjnych. Jednakże biorąc pod uwagę możliwości interpretacji parametrów modelu logitowego wypada on lepiej w tym problemie klasyfikacji.

Model sieci neuronowych był testowany na wielu różnych parametrach odnoszących się do architektury sieci, liczby epok i wielkości batch size. Sprawdzane były również różne algorytmy optymalizacyjne. Wyniki odnoszące się do skuteczności modelu przy różnych konfiguracjach były do siebie bardzo zbliżone. Im więcej warstw ukrytych a także im więcej neuronów w tych warstwach, tym model bardziej dopasowywał się do danych treningowych i gorzej radził sobie na danych testowych. Zbudowany model nie jest zbyt głęboki i nie posiada wielu wartsw ukrytych, ponieważ dane treningowe nie są na tyle duże, aby osiągnąć korzyść z takiej architektury sieci.

### Bibliografia

[Intermediate Topics in Neural Networks](https://towardsdatascience.com/comprehensive-introduction-to-neural-network-architecture-c08c6d8e5d98)


[Simple Guide to Hyperparameter Tuning in Neural Networks](https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594)

[How to Configure the Number of Layers and Nodes in a Neural Network](https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/)

Praca licencjacka - wysłana w załączniku